In [ ]:
!pip install openpyxl

In [238]:
import pandas as pd

import ast
import altair as alt
from tqdm.notebook import tqdm

import numpy as np

In [1]:
course="geomatique_003"
model_results="gpt4o"
model="llama-3.1-70b-versatile"

In [355]:
answers["student"].unique()

array([  123,   230,   528,   784,  2404,  4584,  1908,  5192,  6353,
        6478,  7504,  7967,  8286,  9502,  9749, 12204, 12521, 13359,
       14030, 16920])

In [360]:
# %%
instances_annotation_DSP = [230, 6478, 123, 5192, 16920]
instances_annotation_GEO = [226, 291, 138, 234, 174]
instances_annotation_VILLES = [1090, 2560, 2608, 3170, 1562]


In [361]:
import json

# Specify the file path
file_path = 'data/decomposed_questions.json'

# Read the JSON file
with open(file_path, 'r') as file:
    questions = json.load(file)

# Print the data
print(questions)

{'relevance_selection': ['Is the generated text using the provided data extensively?', 'Is the generated text analysis based largely on the explainer results provided?', 'Is the generated text correctly using the model’s predicted outcome?', 'Is the generated text considering the context (course structure)?', 'Is the generated text selecting the causes that are most relevant to the explainer results?', 'Is the generated text selecting the causes that are most relevant to the question?', 'Is the generated text selecting the causes that are most relevant to the user?', 'Is the generated text selecting some information that is not already shared with the student?'], 'abnormal_conditions': ['Is the generated text using the provided data extensively?', 'Is the generated text analysis based largely on the explainer results provided?', 'Is the generated text correctly using the model’s predicted outcome?', 'Is the generated text considering the context (course structure)?', 'Is the generated 

remember that in abnormal_conditions you have a duplicated question.

In [362]:
lengths = []
for key, item in questions.items():
    lengths.append({key, len(item)})

lengths

[{8, 'relevance_selection'},
 {9, 'abnormal_conditions'},
 {8, 'pearl_explanation'},
 {7, 'necessity_robustness_selection'},
 {9, 'contrastive_explanation'},
 {0, 'hesslow_contrastive'},
 {8, 'base_contrastive_explanation'},
 {5, 'statistical_relevance'},
 {5, 'chain_of_thought'}]

Annotations

relevance 8
abnormal 8
base contrastive 9
cot 5
contrastive 9
necessity 7
pearl 8
statistical 5

In [235]:
# import pandas as pd
# import re

# # Replace 'your_file.xlsx' with the path to your Excel file
# excel_file = 'LLMs_Explainers_Human_Annotation.xlsx'

# # Define the new column names you want to apply
# new_column_names = ['Student Number', 'Explainer', 'Theory', 'Prompt', 'Response'] + \
#                    [f'question_{i+1}' for i in range(9)]  # Adjust range as needed

# # Load the Excel file
# xls = pd.ExcelFile(excel_file)

# # Initialize an empty DataFrame to hold all the data
# combined_df = pd.DataFrame()

# # Loop through each sheet
# for sheet_name in xls.sheet_names:
#     # Load sheet into a DataFrame
#     df = pd.read_excel(xls, sheet_name=sheet_name)
    
#     # Remove columns that have names like "Column 1", "Column 2", etc.
#     df = df.loc[:, ~df.columns.str.match(r'Column \d+')]
    
#     # Rename the columns with the specified list
#     df.columns = new_column_names[:df.shape[1]]
    
#     # Add a column to identify the sheet name
#     df['annotator'] = sheet_name
    
#     # Append to the combined DataFrame
#     combined_df = pd.concat([combined_df, df], ignore_index=True)

# # To save as CSV:
# combined_df.to_csv('all_human_annotations.csv', index=False)

# print("All sheets have been concatenated and saved.")


In [305]:
df=pd.read_csv('all_human_annotations.csv')
df["annotator"]=df["annotator"].str.split("-").str[0]
df.rename(columns={"Student Number":"student"}, inplace=True)
df.rename(columns={"Theory":"strategy"}, inplace=True)
df.rename(columns={"Explainer":"explainer"}, inplace=True)
df.dropna(inplace=True, subset=["student"])
df

,student,explainer,strategy,Prompt,Response,question_1,question_2,question_3,question_4,question_5,question_6,question_7,question_8,annotator,question_9
0,6478.0,LIME,abnormal_conditions,You are an AI assistant that analyzes struggli...,### Report on Student Performance Analysis\n \...,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Davide,NaN
1,6478.0,MC-LIME,abnormal_conditions,You are an AI assistant that analyzes struggli...,### Report on Student Performance Analysis\n \...,Yes,Yes,No,Yes,Yes,Yes,Yes,Yes,Davide,NaN
2,6478.0,CEM,abnormal_conditions,You are an AI assistant that analyzes struggli...,### Student Performance Analysis Report\n \n #...,Yes,Yes,No,Yes,Yes,Yes,Yes,Yes,Davide,NaN
3,230.0,LIME,abnormal_conditions,You are an AI assistant that analyzes struggli...,### Report on Student's Performance in Digital...,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Davide,NaN
4,230.0,MC-LIME,abnormal_conditions,You are an AI assistant that analyzes struggli...,### Analysis Report on Student's Predicted Fai...,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Davide,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22025,174.0,MC-LIME,statistical_relevance,You are an AI assistant that analyzes struggli...,### Student Performance Analysis Report\n \n #...,Yes,Yes,Yes,Yes,Yes,NaN,NaN,NaN,Bhargav,NaN
22026,174.0,CEM,statistical_relevance,You are an AI assistant that analyzes struggli...,### Report on Student Performance Prediction\n...,Yes,Yes,Yes,Yes,Yes,NaN,NaN,NaN,Bhargav,NaN
22027,234.0,LIME,statistical_relevance,You are an AI assistant that analyzes struggli...,### Report on Student's Predicted Performance\...,Yes,Yes,Yes,Yes,Yes,NaN,NaN,NaN,Bhargav,NaN
22028,234.0,MC-LIME,statistical_relevance,You are an AI assistant that analyzes struggli...,### Report on Student Performance Prediction\n...,Yes,Yes,Yes,Yes,Yes,NaN,NaN,NaN,Bhargav,NaN


In [307]:
assert len(df) == 456

In [190]:
#2*3*8*3 + 3*8*13

In [308]:
def map_yes_no_to_binary(df, question_columns):
    """
    Map "Yes" to 1 and "No" to 0 in the specified question columns of the DataFrame.

    Parameters:
    df (pd.DataFrame): The DataFrame containing the data.
    question_columns (list): A list of columns that contain the "Yes"/"No" answers.

    Returns:
    pd.DataFrame: The DataFrame with "Yes"/"No" mapped to 1/0.
    """
    # Map "Yes" to 1 and "No" to 0
    df[question_columns] = df[question_columns].replace({'Yes': 1, 'No': 0})
    return df

df = map_yes_no_to_binary(df, question_columns)


/var/folders/rq/nzqjmf0x07vg4p477yrb9w800000gn/T/ipykernel_60502/3979305336.py:13: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[question_columns] = df[question_columns].replace({'Yes': 1, 'No': 0})


In [309]:
def resolve_duplicates(df, id_columns, question_columns, annotator_column='annotator'):
    """
    Resolve duplicates by calculating the mode for the specified question columns and creating a new row.
    
    Parameters:
    df (pd.DataFrame): The DataFrame containing the data.
    id_columns (list): The columns that identify unique rows ('student', 'strategy', 'explainer').
    question_columns (list): The columns that contain the 0/1 values for which mode needs to be calculated.
    annotator_column (str): The name of the annotator column to set to "everyone" for the new rows.

    Returns:
    pd.DataFrame: The DataFrame with duplicates resolved.
    """
    # Group by the unique identifier columns and find duplicates
    duplicates = df[df.duplicated(subset=id_columns, keep=False)]

    # Create a new DataFrame to hold the resolved duplicates
    resolved_df = pd.DataFrame(columns=df.columns)

    # Iterate over each unique combination of duplicates
    for _, group in duplicates.groupby(id_columns):
        # Calculate the mode for each question column
        mode_values = group[question_columns].mode().iloc[0]
        
        # Create a new row with these mode values
        new_row = group.iloc[0].copy()
        new_row[question_columns] = mode_values
        new_row[annotator_column] = "everyone"
        
        # Append the new row to the resolved DataFrame
        resolved_df = pd.concat([resolved_df, pd.DataFrame([new_row])], ignore_index=True)
    
    # Remove the original duplicates from the DataFrame
    df = df.drop_duplicates(subset=id_columns, keep=False)
    
    # Add the resolved rows to the original DataFrame
    df = pd.concat([df, resolved_df], ignore_index=True)
    
    return df

id_columns = ['student', 'strategy', 'explainer']

# List of question columns
question_columns = ['question_1', 'question_2', 'question_3', 
                    'question_4', 'question_5', 'question_6', 'question_7', 
                    'question_8', 'question_9']

# Resolve duplicates
df = resolve_duplicates(df, id_columns=id_columns, question_columns=question_columns, annotator_column='annotator')

/var/folders/rq/nzqjmf0x07vg4p477yrb9w800000gn/T/ipykernel_60502/1081304849.py:31: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  resolved_df = pd.concat([resolved_df, pd.DataFrame([new_row])], ignore_index=True)
/var/folders/rq/nzqjmf0x07vg4p477yrb9w800000gn/T/ipykernel_60502/1081304849.py:31: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  resolved_df = pd.concat([resolved_df, pd.DataFrame([new_row])], ignore_index=True)


In [365]:
def add_course_column(df, instance_column='instance_id'):
    """
    Add a new column 'course' to the DataFrame, assigning a course name based on the instance IDs.
    
    Parameters:
    df (pd.DataFrame): The DataFrame containing the data.
    instance_column (str): The column name that contains instance IDs. Default is 'instance_id'.

    Returns:
    pd.DataFrame: The DataFrame with a new 'course' column added.
    """
    # Define the course names and corresponding lists of instance IDs
    instances_annotation_dsp_001 = [230, 6478, 123, 5192, 16920]
    instances_annotation_geomatique_003 = [226, 291, 138, 234, 174]
    instances_annotation_villesafricaines_001 = [1090, 2560, 2608, 3170, 1562]
    
    # Define a function to map instance IDs to course names
    def map_instance_to_course(instance_id):
        if instance_id in instances_annotation_dsp_001:
            return "dsp_001"
        elif instance_id in instances_annotation_geomatique_003:
            return "geomatique_003"
        elif instance_id in instances_annotation_villesafricaines_001:
            return "villesafricaines_001"
        else:
            return "unknown"  # Optionally handle unknown instance IDs
    
    # Apply the function to the DataFrame to create the 'course' column
    df['course'] = df[instance_column].apply(map_instance_to_course)
    
    return df

# Example usage:
# Assuming df is your DataFrame and it contains an 'instance_id' column
df = add_course_column(df, instance_column='student')

In [366]:
# List of columns that contain the questions
question_columns = ['question_1', 'question_2', 'question_3', 'question_4', 
                    'question_5', 'question_6', 'question_7', 'question_8', 
                    'question_9']

# Function to calculate the percentage of "Yes" answers
def calculate_yes_percentage(row):
    # Filter out null values
    valid_answers = row[question_columns].dropna()
    # Count "Yes" answers
    yes_count = (valid_answers == 1).sum()
    # Calculate percentage
    if len(valid_answers) == 0:
        return None  # Return None if there are no valid answers
    return (yes_count / len(valid_answers)) * 100

# Apply the function to each row
df['yes_percentage'] = df.apply(calculate_yes_percentage, axis=1)

df

,student,explainer,strategy,Prompt,Response,question_1,question_2,question_3,question_4,question_5,question_6,question_7,question_8,annotator,question_9,yes_percentage,total_1s,course
0,123.0,LIME,abnormal_conditions,You are an AI assistant that analyzes struggli...,### Student Performance Analysis Report\n \n #...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,Davide,NaN,100.0,8.0,dsp_001
1,123.0,MC-LIME,abnormal_conditions,You are an AI assistant that analyzes struggli...,### Analysis Report on Student's Performance P...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,Davide,NaN,100.0,8.0,dsp_001
2,123.0,CEM,abnormal_conditions,You are an AI assistant that analyzes struggli...,### Analysis Report on Student Performance\n \...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,Davide,NaN,100.0,8.0,dsp_001
3,1090.0,LIME,abnormal_conditions,You are an AI assistant that analyzes struggli...,### Student Performance Analysis Report\n \n #...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,Davide,NaN,100.0,8.0,villesafricaines_001
4,1090.0,MC-LIME,abnormal_conditions,You are an AI assistant that analyzes struggli...,### Report on Student Performance Prediction\n...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,Davide,NaN,100.0,8.0,villesafricaines_001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
355,6478.0,LIME,relevance_selection,You are an AI assistant that analyzes struggli...,### Analysis Report on Student Performance Pre...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,everyone,NaN,100.0,8.0,dsp_001
356,6478.0,MC-LIME,relevance_selection,You are an AI assistant that analyzes struggli...,### Report on Student Performance Analysis\n \...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,everyone,NaN,100.0,8.0,dsp_001
357,6478.0,CEM,statistical_relevance,You are an AI assistant that analyzes struggli...,### Analysis Report on Student Performance Pre...,1.0,1.0,0.0,1.0,1.0,NaN,NaN,NaN,everyone,NaN,80.0,4.0,dsp_001
358,6478.0,LIME,statistical_relevance,You are an AI assistant that analyzes struggli...,**Report on Student Performance Prediction in ...,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,everyone,NaN,100.0,5.0,dsp_001


In [367]:
annotation_results=df

## Analysis

In [368]:
courses = ["dsp_001", "geomatique_003", "villesafricaines_001"]

In [369]:
gpt4o_results=pd.read_csv("data/15_students_results/gpt_4_evaluation_all_courses_15_students.csv")

In [370]:
gpt4o_results["student"].unique()

array([  123,   230,  5192,  6478, 16920,   226,   291,   138,   174,
         234,  1090,  2560,  2608,  3170,  1562])

In [371]:
gpt4o_results["strategy"].unique()  

array(['relevance_selection', 'abnormal_conditions', 'pearl_explanation',
       'necessity_robustness_selection', 'contrastive_explanation',
       'base_contrastive_explanation', 'statistical_relevance',
       'chain_of_thought'], dtype=object)

In [372]:
rename_mapping = {f'question_{i}': f'question_{i+1}' for i in range(len(question_columns))}
gpt4o_results = gpt4o_results.rename(columns=rename_mapping)

In [373]:

def adjust_questions_for_abnormal_conditions(df, strategy_column='strategy'):
    # Create a boolean mask for rows where the strategy is "abnormal_conditions"
    abnormal_conditions_mask = df[strategy_column] == "abnormal_conditions"
    
    # Shift the values as specified
    df.loc[abnormal_conditions_mask, 'question_7'] = df.loc[abnormal_conditions_mask, 'question_8']
    df.loc[abnormal_conditions_mask, 'question_8'] = df.loc[abnormal_conditions_mask, 'question_9']
    df.loc[abnormal_conditions_mask, 'question_9'] = np.nan

    return df

# Example usage:
# Assuming df is your DataFrame
gpt4o_results = adjust_questions_for_abnormal_conditions(gpt4o_results)

In [374]:
def calculate_mean_yes_percentage_per_column(df, strategy_column='strategy', yes_percentage_column='yes_percentage'):
    # Calculate the mean yes_percentage for each strategy
    mean_yes_percentage_per_strategy = df.groupby(strategy_column)[yes_percentage_column].mean()

    return mean_yes_percentage_per_strategy

In [375]:
yes_percentage_annotations = calculate_mean_yes_percentage_per_strategy(annotation_results)
yes_percentage_gpt4o = calculate_mean_yes_percentage_per_strategy(gpt4o_results)
print(f"La percentuale di risposte positive delle annotazioni: {yes_percentage_annotations}")
print(f"La percentuale di risposte positive di gpt4o: {yes_percentage_gpt4o}")

La percentuale di risposte positive delle annotazioni: strategy
abnormal_conditions                98.888889
base_contrastive_explanation       97.777778
chain_of_thought                   99.111111
contrastive_explanation            99.012346
necessity_robustness_selection    100.000000
pearl_explanation                  96.111111
relevance_selection                99.722222
statistical_relevance              99.111111
Name: yes_percentage, dtype: float64
La percentuale di risposte positive di gpt4o: strategy
abnormal_conditions               97.037037
base_contrastive_explanation      83.333333
chain_of_thought                  96.000000
contrastive_explanation           99.259259
necessity_robustness_selection    97.777778
pearl_explanation                 99.722222
relevance_selection               98.055556
statistical_relevance             92.444444
Name: yes_percentage, dtype: float64


In [376]:
yes_percentage_annotations_explainer = calculate_mean_yes_percentage_per_strategy(annotation_results, "explainer")
yes_percentage_gpt4o_explainer = calculate_mean_yes_percentage_per_strategy(gpt4o_results, "explainer")
print(f"La percentuale di risposte positive delle annotazioni: {yes_percentage_annotations_explainer}")
print(f"La percentuale di risposte positive di gpt4o: {yes_percentage_gpt4o_explainer}")

La percentuale di risposte positive delle annotazioni: explainer
CEM        98.261574
LIME       98.618056
MC-LIME    99.270833
Name: yes_percentage, dtype: float64
La percentuale di risposte positive di gpt4o: explainer
CEM        95.623016
LIME       96.118056
MC-LIME    94.620040
Name: yes_percentage, dtype: float64


In [377]:
# List of columns that contain the questions
question_columns = ['question_1', 'question_2', 'question_3', 'question_4', 
                    'question_5', 'question_6', 'question_7', 'question_8', 
                    'question_9']

def count_no_answers_per_strategy(df, strategy_column='strategy', question_columns=None):
    """
    Count the number of "No" (0) answers for each question column, grouped by strategy.

    Parameters:
    df (pd.DataFrame): The DataFrame containing the data.
    strategy_column (str): The name of the column containing strategy identifiers. Default is 'strategy'.
    question_columns (list): A list of columns that contain the binary answers (0/1). If None, it will use all columns except the strategy column.

    Returns:
    pd.DataFrame: A DataFrame with strategies as rows and the count of "No" (0) answers per question as columns.
    """
    if question_columns is None:
        # Assume all columns except strategy_column are questions
        question_columns = [col for col in df.columns if col != strategy_column]
    
    # Group by the strategy column and sum the "No" (0) counts for each question
    no_counts_per_strategy = (df[question_columns] == 0).groupby(df[strategy_column]).sum()
    
    return no_counts_per_strategy

# Step 2: Count the "No" (0) answers per strategy
no_counts = count_no_answers_per_strategy(annotation_results, strategy_column='strategy', question_columns=question_columns)

In [378]:
# List of columns that contain the questions
question_columns = ['question_1', 'question_2', 'question_3', 'question_4', 
                    'question_5', 'question_6', 'question_7', 'question_8', 
                    'question_9']

no_counts_questions_annotations=count_no_answers_per_strategy(annotation_results, question_columns=question_columns)
no_counts_questions_annotations

,question_1,question_2,question_3,question_4,question_5,question_6,question_7,question_8,question_9
strategy,,,,,,,,,
abnormal_conditions,0,0,1,0,0,2,0,1,0
base_contrastive_explanation,0,0,1,0,0,5,0,2,0
chain_of_thought,0,0,1,0,1,0,0,0,0
contrastive_explanation,0,0,1,0,0,0,0,2,1
necessity_robustness_selection,0,0,0,0,0,0,0,0,0
pearl_explanation,0,0,0,0,2,2,7,3,0
relevance_selection,0,0,1,0,0,0,0,0,0
statistical_relevance,0,0,1,0,1,0,0,0,0


In [379]:
no_counts_questions_gpt4o=count_no_answers_per_strategy(gpt4o_results, question_columns=question_columns)
no_counts_questions_gpt4o

,question_1,question_2,question_3,question_4,question_5,question_6,question_7,question_8,question_9
strategy,,,,,,,,,
abnormal_conditions,0,0,0,0,0,0,0,0,0
base_contrastive_explanation,0,27,0,17,0,0,0,16,0
chain_of_thought,0,0,0,8,1,0,0,0,0
contrastive_explanation,0,0,0,0,3,0,0,0,0
necessity_robustness_selection,0,0,0,7,0,0,0,0,0
pearl_explanation,0,0,0,0,0,0,1,0,0
relevance_selection,0,0,0,1,0,0,0,6,0
statistical_relevance,0,0,0,17,0,0,0,0,0


In [380]:
def compare_question_columns(df1, df2, id_columns, question_columns):
    """
    Compare two DataFrames row by row based on the identifier columns and question columns.
    
    Parameters:
    df1 (pd.DataFrame): The first DataFrame to compare.
    df2 (pd.DataFrame): The second DataFrame to compare.
    id_columns (list): The columns that identify unique rows ('student_id', 'strategy', 'explainer').
    question_columns (list): The columns that contain the questions to compare.

    Returns:
    pd.DataFrame: A DataFrame with 1 if the question answers match, 0 if they don't,
                  and NaN if all values for a strategy are NaN.
    """
    # Merge the two DataFrames on the identifier columns
    merged_df = pd.merge(df1[id_columns + question_columns], df2[id_columns + question_columns], 
                         on=id_columns, suffixes=('_df1', '_df2'))
    
    # Initialize an empty DataFrame to hold the comparison results
    comparison_df = merged_df[id_columns].copy()
    
    # Compare each question column and store the result as 1 if equal, 0 if not
    for question in question_columns:
        # Handle the comparison, considering NaNs as equal
        comparison_series = (
            (merged_df[f'{question}_df1'] == merged_df[f'{question}_df2']) | 
            (merged_df[f'{question}_df1'].isna() & merged_df[f'{question}_df2'].isna())
        ).astype(int)
        
        # Check if all values in this question for the current strategy are NaN
        all_nan_mask = merged_df[[f'{question}_df1', f'{question}_df2']].isna().all(axis=1)
        
        # Apply the all NaN condition to set NaN where all values are NaN
        comparison_series[all_nan_mask] = np.nan
        
        # Store the result in the comparison DataFrame
        comparison_df[question] = comparison_series
    
    return comparison_df

# Example usage:
# Assuming df1 and df2 are your DataFrames
# Define the columns that identify unique rows
id_columns = ['student', 'strategy', 'explainer']

# List of question columns
question_columns = ['question_1', 'question_2', 'question_3', 'question_4', 
                    'question_5', 'question_6', 'question_7', 'question_8', 
                    'question_9']

# Compare the two DataFrames
comparison_result_df = compare_question_columns(annotation_results, gpt4o_results, id_columns=id_columns, question_columns=question_columns)
comparison_result_df


,student,strategy,explainer,question_1,question_2,question_3,question_4,question_5,question_6,question_7,question_8,question_9
0,123.0,abnormal_conditions,LIME,1,1,1,1,1,1.0,1.0,1.0,NaN
1,123.0,abnormal_conditions,MC-LIME,1,1,1,1,1,1.0,1.0,1.0,NaN
2,123.0,abnormal_conditions,CEM,1,1,1,1,1,1.0,1.0,1.0,NaN
3,1090.0,abnormal_conditions,LIME,1,1,1,1,1,1.0,1.0,1.0,NaN
4,1090.0,abnormal_conditions,MC-LIME,1,1,1,1,1,1.0,1.0,1.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
355,6478.0,relevance_selection,LIME,1,1,1,1,1,1.0,1.0,1.0,NaN
356,6478.0,relevance_selection,MC-LIME,1,1,1,1,1,1.0,1.0,1.0,NaN
357,6478.0,statistical_relevance,CEM,1,1,0,1,1,NaN,NaN,NaN,NaN
358,6478.0,statistical_relevance,LIME,1,1,1,1,1,NaN,NaN,NaN,NaN


In [381]:
def count_mismatches_by_strategy(comparison_df, strategy_column, question_columns):
    """
    Group the comparison DataFrame by strategy and count the number of 0s (mismatches) for each question column.
    If all values for a specific question for a strategy are NaN, then return NaN for that question.

    Parameters:
    comparison_df (pd.DataFrame): The DataFrame containing the comparison results (1s and 0s).
    strategy_column (str): The column name to group by (e.g., 'strategy').
    question_columns (list): The columns that contain the question comparison results (1s and 0s).

    Returns:
    pd.DataFrame: A DataFrame with strategies as rows and the count of 0s per question as columns, or NaN if all values are NaN.
    """
    def count_or_nan(group):
        # Initialize an empty dictionary to store the counts or NaNs
        result = {}
        for question in question_columns:
            # Check if all values for this question are NaN
            if group[question].isna().all():
                result[question] = np.nan
            else:
                # Count the number of 0s (mismatches) if not all are NaN
                result[question] = (group[question] == 0).sum()
        return pd.Series(result)
    
    # Group by strategy and apply the count_or_nan function
    mismatch_counts = comparison_df.groupby(strategy_column).apply(count_or_nan)
    
    return mismatch_counts

# Example usage:
# Assuming comparison_result_df is your DataFrame from the previous comparison
# Define the column to group by
strategy_column = 'strategy'

# List of question columns
question_columns = ['question_1', 'question_2', 'question_3', 'question_4', 
                    'question_5', 'question_6', 'question_7', 'question_8', 
                    'question_9']

# Count the mismatches (0s) by strategy
mismatch_counts_df = count_mismatches_by_strategy(comparison_result_df, strategy_column=strategy_column, question_columns=question_columns)
mismatch_counts_df = mismatch_counts_df.applymap(lambda x: '{:.0f}'.format(x) if pd.notnull(x) else '')

mismatch_counts_df

/var/folders/rq/nzqjmf0x07vg4p477yrb9w800000gn/T/ipykernel_60502/919093104.py:27: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  mismatch_counts = comparison_df.groupby(strategy_column).apply(count_or_nan)
/var/folders/rq/nzqjmf0x07vg4p477yrb9w800000gn/T/ipykernel_60502/919093104.py:43: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  mismatch_counts_df = mismatch_counts_df.applymap(lambda x: '{:.0f}'.format(x) if pd.notnull(x) else '')


,question_1,question_2,question_3,question_4,question_5,question_6,question_7,question_8,question_9
strategy,,,,,,,,,
abnormal_conditions,0,0,1,0,0,2,0,1,
base_contrastive_explanation,0,27,1,17,0,5,0,14,
chain_of_thought,0,0,1,8,2,,,,
contrastive_explanation,0,0,1,0,3,0,0,2,1
necessity_robustness_selection,0,0,0,7,0,0,0,,
pearl_explanation,0,0,0,0,2,2,8,3,
relevance_selection,0,0,1,1,0,0,0,6,
statistical_relevance,0,0,1,17,1,,,,


In [382]:
mismatch_counts_df

,question_1,question_2,question_3,question_4,question_5,question_6,question_7,question_8,question_9
strategy,,,,,,,,,
abnormal_conditions,0,0,1,0,0,2,0,1,
base_contrastive_explanation,0,27,1,17,0,5,0,14,
chain_of_thought,0,0,1,8,2,,,,
contrastive_explanation,0,0,1,0,3,0,0,2,1
necessity_robustness_selection,0,0,0,7,0,0,0,,
pearl_explanation,0,0,0,0,2,2,8,3,
relevance_selection,0,0,1,1,0,0,0,6,
statistical_relevance,0,0,1,17,1,,,,


In [383]:
def percentage_mismatches_by_strategy(comparison_df, strategy_column, question_columns):
    """
    Group the comparison DataFrame by strategy and calculate the percentage of 0s (mismatches)
    for each question column. If all values for a specific question for a strategy are NaN,
    then return NaN for that question. The result is formatted to 2 decimal places.
    
    Parameters:
    comparison_df (pd.DataFrame): The DataFrame containing the comparison results (1s and 0s).
    strategy_column (str): The column name to group by (e.g., 'strategy').
    question_columns (list): The columns that contain the question comparison results (1s and 0s).

    Returns:
    pd.DataFrame: A DataFrame with strategies as rows and the percentage of 0s per question as columns.
    """
    def calculate_percentage_or_nan(group):
        # Initialize an empty dictionary to store the percentages or NaNs
        result = {}
        for question in question_columns:
            # Check if all values for this question are NaN
            if group[question].isna().all():
                result[question] = ""
            else:
                # Count the total number of valid (non-NaN) entries
                total_count = group[question].notna().sum()
                # Count the number of 0s (mismatches)
                mismatch_count = (group[question] == 0).sum()
                # Calculate the percentage
                result[question] = (mismatch_count / total_count * 100).round(2)
        return pd.Series(result)
    
    # Group by strategy and apply the calculate_percentage_or_nan function
    percentage_mismatches = comparison_df.groupby(strategy_column).apply(calculate_percentage_or_nan)
    
    return percentage_mismatches

# Example usage:
# Assuming comparison_result_df is your DataFrame from the previous comparison
# Define the column to group by
strategy_column = 'strategy'

# List of question columns
question_columns = ['question_1', 'question_2', 'question_3', 'question_4', 
                    'question_5', 'question_6', 'question_7', 'question_8', 
                    'question_9']

# Calculate the percentage of mismatches (0s) by strategy
percentage_mismatch_df = percentage_mismatches_by_strategy(comparison_result_df, strategy_column=strategy_column, question_columns=question_columns)
percentage_mismatch_df

/var/folders/rq/nzqjmf0x07vg4p477yrb9w800000gn/T/ipykernel_60502/1549857195.py:32: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  percentage_mismatches = comparison_df.groupby(strategy_column).apply(calculate_percentage_or_nan)


,question_1,question_2,question_3,question_4,question_5,question_6,question_7,question_8,question_9
strategy,,,,,,,,,
abnormal_conditions,0.0,0.0,2.22,0.00,0.00,4.44,0.0,2.22,
base_contrastive_explanation,0.0,60.0,2.22,37.78,0.00,11.11,0.0,31.11,
chain_of_thought,0.0,0.0,2.22,17.78,4.44,,,,
contrastive_explanation,0.0,0.0,2.22,0.00,6.67,0.0,0.0,4.44,2.22
necessity_robustness_selection,0.0,0.0,0.00,15.56,0.00,0.0,0.0,,
pearl_explanation,0.0,0.0,0.00,0.00,4.44,4.44,17.78,6.67,
relevance_selection,0.0,0.0,2.22,2.22,0.00,0.0,0.0,13.33,
statistical_relevance,0.0,0.0,2.22,37.78,2.22,,,,


In [384]:
def overall_percentage_mismatches_by_strategy(comparison_df, strategy_column, question_columns):

    def calculate_overall_percentage(group):
        total_mismatches = 0
        total_valid_counts = 0
        for question in question_columns:
            # Check if all values for this question are NaN
            if not group[question].isna().all():
                # Count the total number of valid (non-NaN) entries
                total_count = group[question].notna().sum()
                # Count the number of 0s (mismatches)
                mismatch_count = (group[question] == 0).sum()
                # Accumulate mismatches and total counts
                total_mismatches += mismatch_count
                total_valid_counts += total_count
        # Calculate the overall percentage
        if total_valid_counts == 0:
            return np.nan  # If all questions were NaN, return NaN
        return (total_mismatches / total_valid_counts * 100).round(2)
    
    # Group by strategy and apply the calculate_overall_percentage function
    overall_percentage_mismatches = comparison_df.groupby(strategy_column).apply(calculate_overall_percentage)
    
    return overall_percentage_mismatches

# Example usage:
# Define the column to group by
strategy_column = 'strategy'

# List of question columns
question_columns = ['question_1', 'question_2', 'question_3', 'question_4', 
                    'question_5', 'question_6', 'question_7', 'question_8', 
                    'question_9']

# Calculate the overall percentage of mismatches across all questions by strategy
overall_percentage_mismatch_df = overall_percentage_mismatches_by_strategy(comparison_result_df, strategy_column=strategy_column, question_columns=question_columns)
overall_percentage_mismatch_df

/var/folders/rq/nzqjmf0x07vg4p477yrb9w800000gn/T/ipykernel_60502/512985608.py:22: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  overall_percentage_mismatches = comparison_df.groupby(strategy_column).apply(calculate_overall_percentage)


strategy
abnormal_conditions                1.11
base_contrastive_explanation      17.78
chain_of_thought                   4.89
contrastive_explanation            1.73
necessity_robustness_selection     2.22
pearl_explanation                  4.17
relevance_selection                2.22
statistical_relevance              8.44
dtype: float64

In [398]:

def calculate_mean_and_std_yes_percentage_per_column(df, column='strategy', yes_percentage_column='yes_percentage'):
    # Group by strategy and calculate the mean and std of yes_percentage
    mean_and_std_yes_percentage_per_strategy = df.groupby(strategy_column)[yes_percentage_column].agg(['mean', 'std'])
    
    # Rename the columns for clarity
    mean_and_std_yes_percentage_per_strategy = mean_and_std_yes_percentage_per_strategy.rename(
        columns={'mean': 'mean_yes_percentage', 'std': 'std_yes_percentage'}
    )

    return mean_and_std_yes_percentage_per_strategy

In [400]:
column = "course"
yes_percentage_column = 'yes_percentage'

# Calculate the mean and std of yes_percentage per strategy
mean_and_std_yes_percentage_df = calculate_mean_and_std_yes_percentage_per_column(annotation_results, column=column, yes_percentage_column=yes_percentage_column)

# Display the result
print(mean_and_std_yes_percentage_df)

                                mean_yes_percentage  std_yes_percentage
strategy                                                               
abnormal_conditions                       98.888889            4.477074
base_contrastive_explanation              97.777778            5.519136
chain_of_thought                          99.111111            4.168182
contrastive_explanation                   99.012346            3.979621
necessity_robustness_selection           100.000000            0.000000
pearl_explanation                         96.111111            6.960839
relevance_selection                       99.722222            1.863390
statistical_relevance                     99.111111            4.168182


In [401]:
column = "course"
yes_percentage_column = 'yes_percentage'

# Calculate the mean and std of yes_percentage per strategy
mean_and_std_yes_percentage_df = calculate_mean_and_std_yes_percentage_per_column(gpt4o_results, column=column, yes_percentage_column=yes_percentage_column)

# Display the result
print(mean_and_std_yes_percentage_df)

                                mean_yes_percentage  std_yes_percentage
strategy                                                               
abnormal_conditions                       97.037037            4.969040
base_contrastive_explanation              83.333333           12.500000
chain_of_thought                          96.000000            8.090398
contrastive_explanation                   99.259259            2.802917
necessity_robustness_selection            97.777778            5.236127
pearl_explanation                         99.722222            1.863390
relevance_selection                       98.055556            5.300324
statistical_relevance                     92.444444            9.806203


# Old analysis functions (might not work with the new code above)

In [58]:
num_questions_per_strategy = {
    'relevance_selection': 8,
    'abnormal_conditions': 9,
    'pearl_explanation': 8,
    'necessity_robustness_selection': 7,
    'contrastive_explanation': 9,
    'base_contrastive_explanation': 8,
    'statistical_relevance': 5
}

# Initialize an empty dictionary to hold the counts for each question
question_counts = {i: [] for i in range(9)}

# Populate the question_counts dictionary
for counts in no_counts_df['no_counts']:
    for i in range(9):
        question_counts[i].append(counts.get(i, 0))

# Create a DataFrame for the long format
long_format_data = []

for question, counts in question_counts.items():
    for i, count in enumerate(counts):
        long_format_data.append({
            'Strategy': no_counts_df['strategy'][i],
            'Question': f'Question {question}',
            'NO Counts': count
        })

long_format_df = pd.DataFrame(long_format_data)

# Remove Question 4
long_format_df = long_format_df[long_format_df['Question'] != 'Question 3']

# Create individual bar charts for each strategy with dynamic x-axis
charts = []
for strategy in long_format_df['Strategy'].unique():
    if strategy == "chain_of_thought":
        continue
    num_questions = num_questions_per_strategy[strategy]
    strategy_df = long_format_df[(long_format_df['Strategy'] == strategy) &
                                 (long_format_df['Question'].apply(lambda x: int(x.split()[-1]) <= num_questions))]
    
    chart = alt.Chart(strategy_df).mark_bar().encode(
        x=alt.X('Question', sort='-y'),
        y=alt.Y('NO Counts', scale=alt.Scale(domain=[0, 60])),
        color='Question',
        tooltip=['Question', 'NO Counts']
    ).properties(
        title=strategy,
        width=300,
        height=300
    )
    charts.append(chart)

# Arrange charts in a grid of 2 columns
rows = []
for i in range(0, len(charts), 3):
    row = alt.hconcat(*charts[i:i+3])
    rows.append(row)

combined_chart = alt.vconcat(*rows)

combined_chart.show()

alt.VConcatChart(...)

In [59]:
no_counts_df.dropna(subset=['strategy'], inplace=True)

In [60]:
# Function to remove the key 3 from the dictionary
def remove_key_3(no_counts):
    if no_counts is None:
        return no_counts
    if 3 in no_counts:
        del no_counts[3]
    return no_counts

# Apply the function to the 'no_counts' column
no_counts_df['no_counts'] = no_counts_df['no_counts'].apply(remove_key_3)

In [61]:
no_counts_df

,strategy,no_counts
0,abnormal_conditions,{}
1,base_contrastive_explanation,"{7: 7, 5: 1, 6: 1}"
2,chain_of_thought,{}
3,contrastive_explanation,"{4: 3, 7: 7, 8: 6}"
4,necessity_robustness_selection,{}
5,pearl_explanation,"{4: 5, 6: 9, 7: 2, 5: 2}"
6,relevance_selection,{7: 14}
7,statistical_relevance,{}


In [62]:
# Function to get the question text and no_counts
def get_question_no_counts(questions, no_counts_df, strategy):
    # Find the corresponding questions for the strategy
    questions = questions[strategy]
    
    # Find the corresponding no_counts for the strategy
    no_counts = no_counts_df[no_counts_df['strategy'] == strategy]['no_counts'].values[0]

    print(no_counts)
    
    # Create the result table
    result = []
    for i in range(len(questions)):
        question_text = questions[i]
        no_count = round(no_counts.get(i, 0) / 58, 2)  # Rounded to 2 decimal places
        result.append([question_text, no_count])
    
    result_df = pd.DataFrame(result, columns=['Question', 'NO Counts percentage'])
    return result_df

# Example usage
strategy = 'statistical_relevance'
result_df = get_question_no_counts(questions, no_counts_df, strategy)

result_df = result_df[result_df['NO Counts percentage'] != 0]
result_df

Counter()


,Question,NO Counts percentage


In [63]:
import plotly.graph_objects as go

# Apply colors based on the percentage using the custom color #DB7F8E
base_color = (219, 127, 142)

def color_scale(value):
    alpha = value  # The value is already a percentage (0 to 1)
    return f'rgba({base_color[0]}, {base_color[1]}, {base_color[2]}, {alpha})'

colors = result_df['NO Counts percentage'].apply(color_scale).tolist()

# Define custom colors
header_color = '#F9DB6D'
cell_color = '#dfedf6'

# Create the Plotly Table with custom colors and increased row height
fig = go.Figure(data=[go.Table(
    header=dict(values=list(result_df.columns[:2]),
                fill_color=header_color,
                align='left',
                height=40),  # Increase header height
    cells=dict(values=[result_df['Question'], result_df['NO Counts percentage']],
               fill_color=[[cell_color]*len(result_df), colors],
               align='left',
               height=30))  # Increase row height
])

fig.show()

In [64]:
questions["relevance_selection"][3]

'Is the generated text considering the context (course structure)?'

In [66]:
# Calculate the percentage of 1s for each strategy, excluding Question 3
# Calculate the percentage of 1s for each strategy, excluding Question 3
def calculate_percentage_of_ones(df):
    result = {}
    strategies = df['strategy'].unique()
    
    for strategy in strategies:
        strategy_df = df[df['strategy'] == strategy]
        total_ones = strategy_df[[f'question_{i}' for i in range(9) if i != 3]].sum().sum()
        total_values = strategy_df[[f'question_{i}' for i in range(9) if i != 3]].count().sum()
        
        if total_values > 0:
            percentage = (total_ones / total_values) * 100
        else:
            percentage = 0
        
        result[strategy] = percentage
    
    return result


# Calculate the percentages
percentages = calculate_percentage_of_ones(evaluation_results)

# Display the percentages
percentages_df = pd.DataFrame(list(percentages.items()), columns=['Strategy', 'Percentage of 1s'])
print(percentages_df)

                         Strategy  Percentage of 1s
0             relevance_selection         89.655172
1             abnormal_conditions         99.137931
2               pearl_explanation         81.280788
3  necessity_robustness_selection         99.137931
4         contrastive_explanation         90.948276
5    base_contrastive_explanation         90.886700
6           statistical_relevance         99.568966


In [67]:
# Calculate the percentage of 1s for each explainer, excluding Question 3
def calculate_percentage_of_ones_by_explainer(df):
    result = {}
    explainers = df['explainer'].unique()
    
    for explainer in explainers:
        explainer_df = df[df['explainer'] == explainer]
        total_ones = explainer_df[[f'question_{i}' for i in range(9) if i != 3]].sum().sum()
        total_values = explainer_df[[f'question_{i}' for i in range(9) if i != 3]].count().sum()
        
        if total_values > 0:
            percentage = (total_ones / total_values) * 100
        else:
            percentage = 0
        
        result[explainer] = percentage
    
    return result

# Calculate the percentages
percentages_by_explainer = calculate_percentage_of_ones_by_explainer(evaluation_results)

# Display the percentages
percentages_df_by_explainer = pd.DataFrame(list(percentages_by_explainer.items()), columns=['Explainer', 'Percentage of 1s'])
print(percentages_df_by_explainer)

  Explainer  Percentage of 1s
0      LIME         92.872340
1   MC-LIME         93.026005
2       CEM         91.595745
